# 1. rubot Mecanum model description

Diferent models could be obtained:
- Commercial robot nexus robot
- Costomized nexus robot
- Custom designed model 

For a custom model created in SolidWorks, there is an add-in that allows the conversion of SW Parts and Assemblies into a URDF file. The exporter will create a ROS-like package that contains a directory for meshes, textures and robots (urdf files).: http://wiki.ros.org/sw_urdf_exporter




## 1.1. Commercial nexus robot

This commercial robot can be found in: https://www.nexusrobot.com/product/4wd-mecanum-wheel-mobile-arduino-robotics-car-10011.html

There is already a URDF model extracted from: https://github.com/RBinsonB/nexus_4wd_mecanum_simulator

Run the simulation for one nexus robot by using the following command line:

In [ ]:
roslaunch nexus_4wd_mecanum_gazebo nexus_4wd_mecanum_world.launch

![Getting Started](./Images/1_rubot_gazebo1.png)

To launch two nexus robots, use the following launch file:

In [ ]:
roslaunch nexus_4wd_mecanum_gazebo nexus_4wd_mecanum_multi.launch

![Getting Started](./Images/1_rubot_gazebo2.png)

## 1.2. Costomized nexus robot

We have created a new "nexus_mecanum" package in robot description folder.
In the URDF file we have added:
- Camera sensor
- LIDAR sensor
- Kinematic driver for Holonomic movement


### Camera sensor

This sensor is integrated as a link and fixed joint for visual purposes:

In [ ]:
  <!-- 2D Camera as a mesh of actual PiCamera -->
  <link name="camera">
    <visual>
      <origin rpy="0 1.570795 0" xyz="0 0 0"/>
      <geometry>
        <mesh filename="package://nexus_4wd_mecanum_description/meshes/piCamera.stl" scale="0.0025 0.0025 0.0025"/>
      </geometry>
      <material name="yellow"/>
    </visual>
    <collision>
      <origin rpy="0 1.570795 0" xyz="0 0 0"/>
      <geometry>
        <box size="0.075 0.075 0.025"/>
      </geometry>
    </collision>
    <inertial>
      <origin rpy="0 1.570795 0" xyz="0 0 0"/>
      <mass value="1e-3"/>
      <inertia ixx="1e-6" ixy="0" ixz="0" iyy="1e-6" iyz="0" izz="1e-6"/>
    </inertial>
  </link>
  <!-- 2D Camera JOINT base_link -->
  <joint name="joint_camera" type="fixed">
    <axis xyz="0 0 1"/>
    <origin rpy="0 0 0" xyz="0.16 0 0.05"/>
    <parent link="base_link"/>
    <child link="camera"/>
  </joint>

A driver is needed to view the images.

In [ ]:
  <!-- 2D Camera controller -->
  <gazebo reference="camera">
    <sensor name="camera1" type="camera">
      <update_rate>30.0</update_rate>
      <camera name="front">
        <horizontal_fov>1.3962634</horizontal_fov>
        <image>
          <width>800</width>
          <height>800</height>
          <format>R8G8B8</format>
        </image>
        <clip>
          <near>0.02</near>
          <far>300</far>
        </clip>
      </camera>
      <plugin filename="libgazebo_ros_camera.so" name="camera_controller">
        <alwaysOn>true</alwaysOn>
        <updateRate>0.0</updateRate>
        <cameraName>rubot/camera1</cameraName>
        <imageTopicName>image_raw</imageTopicName>
        <cameraInfoTopicName>camera_info</cameraInfoTopicName>
        <frameName>camera</frameName>
        <hackBaseline>0.07</hackBaseline>
        <distortionK1>0.0</distortionK1>
        <distortionK2>0.0</distortionK2>
        <distortionK3>0.0</distortionK3>
        <distortionT1>0.0</distortionT1>
        <distortionT2>0.0</distortionT2>
      </plugin>
    </sensor>
  </gazebo>

### LIDAR sensor

This sensor is integrated as a link and fixed joint for visual purposes:

In [ ]:
  <!-- LIDAR base_scan -->
  <link name="base_scan">
    <visual name="sensor_body">
      <origin rpy="0 0 0" xyz="0 0 0.04"/>
      <geometry>
        <mesh filename="package://nexus_4wd_mecanum_description/meshes/X4.stl" scale="0.0015 0.0015 0.0015"/>
      </geometry>
      <material name="yellow"/>
    </visual>
    <collision>
      <origin rpy="0 0 0" xyz="0 0 0.04"/>
      <geometry>
        <cylinder length="0.01575" radius="0.0275"/>
      </geometry>
    </collision>
    <inertial>
      <origin rpy="0 0 0" xyz="0 0 0.4"/>
      <mass value="0.057"/>
      <inertia ixx="0.001" ixy="0.0" ixz="0.0" iyy="0.001" iyz="0.0" izz="0.001"/>
    </inertial>
  </link>
  <!-- LIDAR base_scan JOINT base_link -->
  <joint name="scan_joint" type="fixed">
    <axis xyz="0 0 1"/>
    <origin rpy="0 0 0" xyz="0 0 0.09"/>
    <parent link="base_link"/>
    <child link="base_scan"/>
  </joint>

A driver is needed to see the 720 laser distance points:

In [ ]:
  <!-- Laser Distance Sensor YDLIDAR X4 controller-->
  <gazebo reference="base_scan">
    <sensor name="lds_lfcd_sensor" type="ray">
      <pose>0 0 0 0 0 0</pose>
      <visualize>false</visualize>
      <update_rate>5</update_rate>
      <ray>
        <scan>
          <horizontal>
            <samples>720</samples>
            <resolution>0.5</resolution>
            <min_angle>0.0</min_angle>
            <max_angle>6.28319</max_angle>
          </horizontal>
        </scan>
        <range>
          <min>0.12</min>
          <max>10</max>
          <resolution>0.015</resolution>
        </range>
        <noise>
          <type>gaussian</type>
          <!-- Noise parameters based on published spec for YDLIDAR X4
              is 1.5% at half range 4m (= 60mm, "+-160mm" accuracy at max. range 8m).
              A mean of 0.0m and stddev of 0.020m will put 99.7% of samples
              within 0.16m of the true reading. -->
          <mean>0.0</mean>
          <stddev>0.02</stddev>
        </noise>
      </ray>
      <plugin filename="libgazebo_ros_laser.so" name="gazebo_ros_lds_lfcd_controller">
        <!-- topicName>/gopigo/scan</topicName -->
        <topicName>scan</topicName>
        <frameName>base_scan</frameName>
      </plugin>
    </sensor>
  </gazebo>

### Mecanum Drive controller
A driver is needed to describe the kinematics.This kinematics is described in the "libgazebo_ros_planar_move.so" file and the URDF model will contain the specific gazebo plugin.

This driver is the "Planar Move Plugin" and is described in Gazebo tutorials: http://gazebosim.org/tutorials?tut=ros_gzplugins#AddingaModelPlugin

In [ ]:
  <!-- Mecanum drive controller -->
  <gazebo>
    <plugin name="Mecanum_controller" filename="libgazebo_ros_planar_move.so">
      <commandTopic>cmd_vel</commandTopic>
      <odometryTopic>odom</odometryTopic>
      <odometryFrame>odom</odometryFrame>
      <odometryRate>50.0</odometryRate>
      <robotBaseFrame>base_footprint</robotBaseFrame>
    </plugin>
  </gazebo>

We can open the new model in rviz and gazebo:

roslaunch nexus_mecanum display.launch

roslaunch nexus_mecanum gazebo.launch

![Getting Starter](./Images/1_nexus_mecanum2.png)

To control the POSE with cmd_vel we can install the package: 
- For Holonomic movements: http://wiki.ros.org/teleop_twist_keyboard
- for non-holonomic movements: http://wiki.ros.org/teleop_tools

In [ ]:
sudo apt-get install ros-melodic-teleop-tools
or
sudo apt-get install ros-melodic-teleop-twist-keyboard

To launch the control:

In [ ]:
rosrun key_teleop key_teleop.py /key_vel:=/cmd_vel
or
rosrun teleop_twist_keyboard teleop_twist_keyboard.py

### 1.3. rUBot mecanum model

We can create a new model in 3D using SolidWorks and use the URDF plugin to generate the URDF file model: rubot_mecanum.urdf

This model is located in a new "rubot_mecanum" package

We add the same sensors and plugins.

We can open the new model in rviz and gazebo:

roslaunch rubot_mecanum display.launch

roslaunch rubot_mecanum gazebo.launch

![Getting Starter](./Images/1_rubot_mecanum2.png)

## 2. Mecanum robot spawn in world
We have created a specific world "square.world" and we can spawn the robot model inside this world.

For that, we have created a "nexus_world.launch" file.

Type to launch the file:

roslaunch nexus_mecanum nexus_world.launch
roslaunch nexus_mecanum display.launch

In [ ]:
<?xml version="1.0"?>
<launch>
  <!-- Define the needed parameters -->
  <arg name="world" default="square.world"/> 
  <arg name="model" default="nexus.urdf" />
  <arg name="x_pos" default="0.5"/>
  <arg name="y_pos" default="0.5"/>
  <arg name="z_pos" default="0.0"/>

  <include file="$(find gazebo_ros)/launch/empty_world.launch">
    <arg name="world_name" value="$(find nexus_mecanum)/worlds/$(arg world)" />
  </include>
  <!-- Spawn the robot -->
  <!-- Robot URDF definition -->
    <param name="robot_description" textfile="$(find nexus_mecanum)/urdf/$(arg model)"/>
  <node name="spawn_model" pkg="gazebo_ros" type="spawn_model"
    args="-urdf -model nexus -param robot_description -x $(arg x_pos) -y $(arg y_pos) -z $(arg z_pos)" />
</launch>


In the case of nexus_mecanum robot:

![Getting Starter](./Images/1_nexus_mecanum3.png)

In the case of rubot_mecanum robot:

![Getting Starter](./Images/1_rubot_mecanum3.png)

Carefull!

There is a problem in lidar position and no camera images